In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Download the image assets from drive

In [0]:
# Background Images
!cp -r /content/gdrive/My\ Drive/Assignment15/A/Input/bg  /content/
# Foreground Images
!cp -r /content/gdrive/My\ Drive/Assignment15/A/Input/fg150  /content/
# Foreground Masks
!cp -r /content/gdrive/My\ Drive/Assignment15/A/Input/fg_mask  /content/

## Overlay FG over BG (Generate FG_BG images and it's Masks)

In [0]:
from random import randint
import cv2
import numpy as np

def makeCombinations( background, foreground, fg_mask, opDir, imageNo, labelFile, labelInfo):

  ratio = 1.0
  bg_wid, bg_hei = background.size
  for j in range(4):
    fg_wid, fg_hei = foreground.size
    fg_wid  = fg_wid * ratio
    fg_hei  = fg_hei * ratio
    fg_resized   = foreground.resize((int(fg_wid),int(fg_hei)), Image.ANTIALIAS)
    mask_resized = fg_mask.resize((int(fg_wid),int(fg_hei)), Image.ANTIALIAS)
    for i in range(0,5):
      labelData = labelInfo.copy()
      bg_copy = background.copy()
      pos_x = np.random.randint(0, bg_wid - fg_wid, size=1)
      pos_y = np.random.randint(0, bg_hei - fg_hei, size=1)
      bg_copy.paste(fg_resized, (pos_x, pos_y), fg_resized)
      
      bg_mask = Image.new('L', bg_copy.size)
      fg_mask = fg_mask.convert('L')
      bg_mask.paste(mask_resized, (pos_x, pos_y), mask_resized)
      # save the background and the mask as temp .jpg files    
      imageNo = imageNo + 1
      bg_copy.save(opDir+'/overlay/'+str(imageNo)+".jpg", quality=50, optimize=True)
      bg_mask.save(opDir+'/mask/'+str(imageNo)+".jpg", quality=50, optimize=True)

      labelData.append(opDir+'/overlay/'+str(imageNo)+".jpg")
      labelData.append(opDir+'/mask/'+str(imageNo)+".jpg")
      labelData.append(opDir+'/depth/'+str(imageNo)+".jpg")
      labelFile.write(labelData)

    ratio = ratio - 0.2
    

## CSVWriter class to create the Label Data

In [0]:
import csv
class CSVWriter():

    filename = None
    fp = None
    writer = None

    def __init__(self, filename):
        self.filename = filename
        self.fp = open(self.filename, 'w', encoding='utf8')
        self.writer = csv.writer(self.fp, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL, lineterminator='\n')

    def close(self):
        self.fp.close()

    def write(self, elems):
        self.writer.writerow(elems)

    def size(self):
        return os.path.getsize(self.filename)

    def fname(self):
        return self.filename

## Create Overlay and Mask ( Main program)

In [0]:
!rm -r Dataset

rm: cannot remove 'Dataset': No such file or directory


In [0]:
from os import listdir

# from google.colab.patches import cv2_imshow
# import cv2
import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
import time

%matplotlib inline
import os, errno

# Paths used
DB_name = 'Dataset/'
path_BG = 'bg'
path_FG = 'fg150'
path_fg_mask = 'fg_mask'
bg_imageListDir = listdir(path_BG)
fg_imageListDir = listdir(path_FG)

try:
    os.makedirs(DB_name + "bg")
except FileExistsError:
    pass

# All dataset info will save to "label_data.csv"
dataList = CSVWriter(DB_name + 'Dataset/label_data.csv')

program_starts = time.time()
for bgID,bg_image in enumerate(bg_imageListDir):
  now = time.time()
  print("iter = {}, bg_mage = {}, {} seconds since the loop started".format(bgID,bg_image, (now - program_starts)))
  outputDir = DB_name
  outputDir = outputDir + bg_image[:-4]
  bg_img = Image.open(path_BG+'/'+bg_image).resize((224,224), Image.ANTIALIAS)
  bg_img.save( DB_name + "bg/" + bg_image, quality=60, optimize=True)
  labelInfo = [DB_name + "bg/" + bg_image]
  for fg_image in fg_imageListDir:
    outputDir_fg = outputDir + '/' + fg_image[:-4]
    fg_img  = Image.open(path_FG+'/'+fg_image)
    fg_mask = Image.open(path_fg_mask+'/mask_'+fg_image)

    try:
        os.makedirs(outputDir_fg + "/overlay")
        os.makedirs(outputDir_fg + "/mask")
        os.makedirs(outputDir_fg + "/depth")
    except FileExistsError:
        pass
    
    makeCombinations( bg_img, fg_img, fg_mask, outputDir_fg, 0, dataList, labelInfo)
    # Flip foreground image and its mask horizontally
    fg_img = fg_img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
    fg_mask = fg_mask.transpose(PIL.Image.FLIP_LEFT_RIGHT)

    makeCombinations( bg_img, fg_img, fg_mask, outputDir_fg, 20, dataList, labelInfo)
print("{} seconds since the loop started".format((now - program_starts)))
dataList.close()


iter = 0, bg_mage = bg80.jpg, 0.00038695335388183594 seconds since the loop started
iter = 1, bg_mage = bg69.jpg, 13.104660034179688 seconds since the loop started
iter = 2, bg_mage = bg20.jpg, 26.017130136489868 seconds since the loop started
iter = 3, bg_mage = bg27.jpg, 38.89763927459717 seconds since the loop started
iter = 4, bg_mage = bg84.jpg, 51.682793855667114 seconds since the loop started
iter = 5, bg_mage = bg99.jpg, 64.9380042552948 seconds since the loop started
iter = 6, bg_mage = bg28.jpg, 78.6138060092926 seconds since the loop started
iter = 7, bg_mage = bg79.jpg, 92.01183319091797 seconds since the loop started
iter = 8, bg_mage = bg13.jpg, 105.31782937049866 seconds since the loop started
iter = 9, bg_mage = bg39.jpg, 118.10681676864624 seconds since the loop started
iter = 10, bg_mage = bg7.jpg, 131.70175695419312 seconds since the loop started
iter = 11, bg_mage = bg72.jpg, 144.62469172477722 seconds since the loop started
iter = 12, bg_mage = bg87.jpg, 157.245217

### Compress Dataset folder to zip format

In [0]:
import shutil
shutil.make_archive('Dataset', 'zip', 'Dataset')
#! unzip -q zip_file_name.zip

'/content/Dataset.zip'

### Copy compressed Dataset.zip file to Google drive

In [0]:
!cp -r /content/Dataset.zip /content/gdrive/My\ Drive/Assignment15/A/Output